In [ ]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import warnings
import pprint
import time
warnings.simplefilter('ignore')

#ファイルのインポート
import evaluation as ev
import control_structure as cs
import race
import table as tb
import init_table as init
import creating_test as test
import os_operation as _os_

RESULTS_FILE_NAME='sample' #.csvつけない

# インポートする dir
path='data_buffer2/'

# エクスポートする dir
results_path='results_buffer2/'


files=_os_.get_files(path)
send_files=files['send_files']
recv_files=files['recv_files']


heading_results=('Task','Buffer','TestCase','SVAR','Def-Use','Plnv')
heading=('Thread','Port','Event','Index')
pre_results=pd.DataFrame({},columns=heading_results)
new_results=pd.DataFrame({},columns=heading_results)

for file in tqdm(range(len(recv_files))):
    print('-------{}回目-------'.format(file+1))
    RECEIVE_SHEET_NAME=path+recv_files[file]
    SEND_SHEET_NAME=path+send_files[file]
    
    Qs=[pd.DataFrame({})]
    Qr=[pd.DataFrame({})]
    Qr[0]=pd.read_csv(RECEIVE_SHEET_NAME,names=heading)
    Qs[0]=pd.read_csv(SEND_SHEET_NAME,names=heading)
    
    init_tables=init.init_main(Qr,Qs)
    Qr=init_tables['Qr']
    Qs=init_tables['Qs']
    Qr_unique=init_tables['Qr_unique']
    Qs_unique=init_tables['Qs_unique']

    for way in range(2):
        if way==0:
            t_way=2
            combination=0 #0の場合は既存研究の方法、1の場合は組み合わせを考慮した手法
            ev_table=pre_results
        elif way==1:
            t_way=2
            combination=1
            ev_table=new_results
            Qr=Qr[:1]
            Qs=Qs[:1]
        else:
            print('プログラムを終了します') # shoudn't reach here
            break


        number=0 # 実験の回数
        check_digit=0

        race_set={}


        start = time.time()
        print("Creating race set",end='')
        race_set=race.creating_race_set(Qr,Qs,Qs_unique,race_set,combination)
        elapsed_time = time.time() - start
        print ("\nCreating race set took:{:.4g}".format(elapsed_time) + "[sec]")
        #pprint.pprint((race_set))


        start = time.time()
        print("Creating race table",end='')
        table=tb.construct_race_table(Qs,Qr,race_set,t_way)
        if t_way>1:
            table=tb.expand_table(Qr,Qs,race_set,table,t_way)
        elapsed_time = time.time() - start
        print ("\nCreating race table took:{:.4g}".format(elapsed_time) + "[sec]")
        table=table.astype('int64')

        check_digit=1

        start = time.time()
        results=test.create_new_testcase(number,check_digit,Qr,Qs,table,race_set,Qr_unique,Qs_unique)
        Qr=results['recv']
        Qs=results['send']
        elapsed_time = time.time() - start
        print ("Creating test case took:{:.4g}".format(elapsed_time) + "[sec]")
        print('The number of Test Case is {}.'.format(len(Qr)))


        evaluation=ev.evaluate(Qr,Qs)
        Q=pd.concat([Qs[0],Qr[0]]).sort_index()
        display(table)
        if way==0:
            pre_results=pre_results.append(evaluation,ignore_index=True)
        elif way==1:
            new_results=new_results.append(evaluation,ignore_index=True)
        else:
            print('error')
        
_os_.save_file(RESULTS_FILE_NAME,results_path,pre_results,new_results)


  0%|          | 0/10 [00:00<?, ?it/s]

-------1回目-------
Creating race set
Creating race set took:0.06042[sec]
Creating race table.Expanding table

Creating race table took:0.004445[sec]
Creating test case took:0[sec]
The number of Test Case is 1.


""


Creating race set
Creating race set took:0.05806[sec]
Creating race table.Expanding table

Creating race table took:0.007674[sec]
Creating test case took:0[sec]
The number of Test Case is 1.


""


 10%|█         | 1/10 [00:06<00:59,  6.61s/it]

-------2回目-------
Creating race set
Creating race set took:0.423[sec]
Creating race table................Expanding table

Creating race table took:5.537[sec]
Creating test case took:437.5[sec]
The number of Test Case is 25.


In [33]:
import numpy as np
    for i in range(len(Qs[0])):
        if Qr[0].iloc[i].Port==Qs[0].iloc[i].Port:
            pass
        else: #一致しなかった場合
            port=Qs[0].iloc[i].Port
            df=Qr[0][i+1:]
            if len(df)!=0:
                continue
            else:
                new_index=np.arange(len(Qr[0]))
                new_pair_index=df[df.Port==port].iloc[0].name
                new_index[i]=new_pair_index
                new_index[new_pair_index]=i
                Qr[0]=Qr[0].set_index('new_index')
                Qr[0].sort_index(inplace=True)

In [34]:
Qr[0]

,ID,Thread,Port,Event,Index
0,r1,T10,6476868,recv,1
1,r2,T1,6476868,recv,1
2,r3,T2,6476868,recv,1
3,r4,T3,6476868,recv,1
4,r5,T5,6476868,recv,1
...,...,...,...,...,...
76,r77,T8,6478964,recv,8
77,r78,T3,6478964,recv,8
78,r79,T4,6478964,recv,8
79,r80,T9,6478964,recv,8


In [5]:
Qr[number+1][Qr[number+1]['ID']==columns[key]]

,ID,Thread,Port,Event,Index


In [27]:
Qr[0]['new_index']=new_index

In [28]:
Qr[0]

,ID,Thread,Port,Event,Index,new_index
0,r1,T10,6476868,recv,1,0
1,r2,T1,6476868,recv,1,1
2,r3,T2,6476868,recv,1,2
3,r4,T3,6476868,recv,1,3
4,r5,T5,6476868,recv,1,4
...,...,...,...,...,...,...
76,r77,T8,6478964,recv,8,76
77,r78,T3,6478964,recv,8,77
78,r79,T4,6478964,recv,8,78
79,r80,T9,6478964,recv,8,79


['recv-task-1-buffer-2.csv',
 'recv-task-10-buffer-2.csv',
 'recv-task-2-buffer-2.csv',
 'recv-task-2-loop-5.csv',
 'recv-task-3-buffer-2.csv',
 'recv-task-4-buffer-2.csv',
 'recv-task-5-buffer-2.csv',
 'recv-task-6-buffer-2.csv',
 'recv-task-7-buffer-2.csv',
 'recv-task-8-buffer-2.csv',
 'recv-task-9-buffer-2.csv']

In [20]:
pd.set_option('display.max_rows', 900)

,r1,r2,r9,r10,r11,r12,r19,r20,r21,r22,r29,r30,r31,r32
0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,2,0,1,0,1,0,1,0,1,0,1,0,1
2,0,3,0,0,1,0,1,0,1,0,1,0,1,0
3,1,0,1,0,0,1,1,0,0,1,1,0,0,1
4,1,1,0,0,0,0,0,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,0,0,0,0,0,2
939,0,0,0,0,0,0,0,0,0,0,0,0,4,0
940,0,0,0,0,0,0,0,0,0,0,0,0,4,1
941,0,0,0,0,0,0,0,0,0,0,0,0,2,3


In [27]:
table[table['r2']==1]

,r1,r2,r9,r10,r11,r12,r19,r20,r21,r22,r29,r30,r31,r32
0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,0,0,0,0,0,1,1,0,0,1,1,0
8,2,1,1,0,0,1,1,0,0,0,0,0,0,0
12,3,1,1,1,1,1,1,1,1,1,1,0,0,1
16,4,1,1,1,1,1,1,1,1,1,1,1,1,1
32,0,1,2,0,0,0,0,0,0,0,0,0,0,0
39,0,1,3,0,0,0,0,0,0,0,0,0,0,0
58,0,1,-1,2,0,0,0,0,0,0,0,0,0,0
60,0,1,-1,3,0,0,0,0,0,0,0,0,0,0
88,0,1,-1,0,2,0,0,0,0,0,0,0,0,0


In [17]:
race_set

{'r1': ['s8', 's18', 's28', 's38'],
 'r2': ['s9', 's19', 's29'],
 'r3': [],
 'r4': [],
 'r5': [],
 'r6': [],
 'r7': [],
 'r8': [],
 'r9': ['s42'],
 'r10': ['s39'],
 'r11': ['s40'],
 'r12': ['s41'],
 'r13': [],
 'r14': [],
 'r15': [],
 'r16': [],
 'r17': [],
 'r18': [],
 'r19': ['s42'],
 'r20': ['s39'],
 'r21': ['s40'],
 'r22': ['s41'],
 'r23': [],
 'r24': [],
 'r25': [],
 'r26': [],
 'r27': [],
 'r28': [],
 'r29': ['s42'],
 'r30': ['s39'],
 'r31': ['s40'],
 'r32': ['s41'],
 'r33': [],
 'r34': [],
 'r35': [],
 'r36': [],
 'r37': [],
 'r38': [],
 'r39': [],
 'r40': [],
 'r41': [],
 'r42': []}

In [108]:
combination_table.iloc[0]

0

In [55]:
Qs_unique

,ID,Thread,Port,Event,Index,cstruct
0,s1,T1,p2,send,1,
1,s2,T3,p2,send,1,
2,s3,T4,p2,send,1,
3,s4,T1,p2,send,2,[s1]
4,s5,T1,p2,send,3,"[s1, s4]"


In [9]:
import evaluation as ev

In [10]:
ev.hello()

hello


In [29]:
_os_.save_file(RESULTS_FILE_NAME,results_path,pre_results,new_results)

In [47]:
def cstruct(event,results):
    global Q
    global Qs
    global Qr
    global check_digit
    
    if check_digit==0:
        magic=0
    else:
        magic=1
    
    thread=event['Thread']
    index=event['Index']
       
    if index==1:
        return ''
    for i in range(index-1,0,-1):
        try:
            temp=Qr[number+magic][(Qr[number+magic].Thread == thread) & (Qr[number+magic].Index==i)].iloc[0]
        except:
            temp=Qs[number+magic][(Qs[number+magic].Thread == thread) & (Qs[number+magic].Index==i)].iloc[0]
        if temp['Event']=='send':
            results.append(temp['ID'])
            print('true:{}'.format(temp['ID']))
            event=temp
            results.extend(cstruct(event,results))
        else:
            results.append(temp['ID'])
            send=Qs[number+magic].iloc[temp.name]
            results.append(send['ID'])
            print('else:{}'.format(send['ID']))
            results.extend(cstruct(send,results))
    return list(set(results))


In [48]:
i=0
check_digit=0

In [12]:
Qr_unique

,ID,Thread,Port,Event,Index,coler,cstruct
0,r1,T2,[p2],res,1,white,
1,r2,T2,[p2],res,2,white,"[s1, r1]"
2,r3,T3,[p3],res,1,white,
3,r4,T3,[p3],res,2,white,"[r3, s3, s2, r2, s1, r1]"
4,r5,T3,[p3],res,2,white,"[r3, s4, s2]"
5,r6,T2,[p2],res,2,white,"[s2, r1]"
6,r7,T3,[p3],res,2,white,"[r3, r6, s3, s2, s1, r1]"


In [40]:
table

,r1,r3
0,0,1
1,1,0
2,1,1
